<a href="https://colab.research.google.com/github/satann3456/DQN_mario/blob/master/mario_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so

!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip install pyglet
!pip install pyopengl
!pip install gym[classic_control]
!pip install keras-rl
!pip install gym-super-mario-bros

E: Unable to locate package libcusparse8.0
E: Couldn't find any package by glob 'libcusparse8.0'
E: Couldn't find any package by regex 'libcusparse8.0'
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
    100% |████████████████████████████████| 40kB 5.8MB/s 
  Running setup.py bdist_wheel for keras-rl ... - \ done
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
    100% |████████████████████████████████| 204kB 27.8MB/s 
    100% |████████████████████████████████| 12.1MB 2.3MB/s 
    100% |████████████████████████████████| 81kB 28.7MB/s 
  Running setup.py bdist_wheel for nes-py ... - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/3c/a3/f8/56f889fbb151951c518797e32f630cd944d197a3e16fb16415
Successfully built nes-py


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools > /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse > /dev/null

from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


### 報酬獲得方法を変更する場合

/usr/local/lib/python3.6/dist-packages/gym_super_mario_brosにあるsmb_env.pyをダウンロードして中の報酬値の設定を変更する

（報酬値の変更の仕方はsmb_env.pyのコード読めばわかるはず）

そして上書き更新する

colaboratoryが再起動するたびに実行しないと報酬値が混ざるので注意

In [0]:
!ls /usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/

actions.py  __init__.py  _registration.py  smb_env.py
_app	    __pycache__  _roms		   tests


In [0]:
!cp smb_env.py /usr/local/lib/python3.6/dist-packages/gym_super_mario_bros

### トレーニング

In [0]:
import numpy as np
import gym
import keras

from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

#動画や報酬値の結果を保存したいフォルダを選択。毎回変えないと前のデータが消えるので注意
folder = './drive/ProX/mario7_log21'

#ここを変えればgym_super_mario_brosのEnvironmentsを変更できる。種類はgym_super_mario_brosのgithub参照
ENV_NAME = 'SuperMarioBros-v0' 

INPUT_SHAPE = (84, 84) #入力画像のサイズを設定
WINDOW_LENGTH = 4 

# 入力画像をグレースケールにしたり、INPUT SHAPEの値にサイズを変更
# また、報酬値を1 ∼ -1の値に正規化する
class GameProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

# ビデオを保存する
# intervalの値を変更すれば動画を保存するエピソードの間隔を設定できる
interval = 10
env = wrappers.Monitor(env, folder, force=True, video_callable=(lambda ep: ep % interval == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2,3,1),input_shape=input_shape))
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = GameProcessor()

# εグリーディ法を使用する。epsが高いとランダム動作をとる確率が高くなるので徐々に低い値にする(0∼1の範囲)
# 他の手法もあるのでkeras-rlのgithubのページ参照
policy = EpsGreedyQPolicy(eps=.9)
#policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.,value_test=.05, value_min=.1, nb_steps=400000)

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=500, 
             processor=processor, target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

#dqn.load_weights('./drive/ProX/dqn_eps2_{}_weights.h5f'.format(ENV_NAME))
checkpoint_weights_filename = folder + '/dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = folder + '/dqn_{}_log.json'.format(ENV_NAME)

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env,callbacks=callbacks, nb_steps=500000, visualize=False, verbose=2)
#dqn.save_weights('./drive/ProX/dqn_eps2_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
acti

### モデルの画像化

In [0]:
!pip install graphviz

In [0]:
!pip install pydot_ng

In [0]:
import numpy as np
import gym
import keras

from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

#動画や報酬値の結果を保存したいフォルダを選択。毎回変えないと前のデータが消えるので注意
folder = './drive/ProX/mario7_log21'

#ここを変えればgym_super_mario_brosのEnvironmentsを変更できる。種類はgym_super_mario_brosのgithub参照
ENV_NAME = 'SuperMarioBros-v0' 

INPUT_SHAPE = (84, 84) #入力画像のサイズを設定
WINDOW_LENGTH = 4 

# 入力画像をグレースケールにしたり、INPUT SHAPEの値にサイズを変更
# また、報酬値を1 ∼ -1の値に正規化する
class GameProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

# ビデオを保存する
# intervalの値を変更すれば動画を保存するエピソードの間隔を設定できる
interval = 10
env = wrappers.Monitor(env, folder, force=True, video_callable=(lambda ep: ep % interval == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2,3,1),input_shape=input_shape))
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

plot_model(model, to_file='model.png')


ModuleNotFoundError: ignored

# 試行錯誤したコード

### テスト

In [0]:
import numpy as np
import gym
import keras
#import pydot

from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

from keras import backend as K # 柴田


ENV_NAME = 'SuperMarioBros-v0'

INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class GameProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, './drive/ProX/tst/mario_log15', force=True, video_callable=(lambda ep: ep % 1 == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)


input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add( Permute((2,3,1), input_shape=input_shape ) )
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())


memory = SequentialMemory(limit=30000, window_length=WINDOW_LENGTH)
processor = GameProcessor()
policy = EpsGreedyQPolicy(eps=.05)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=500, 
             processor=processor, target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=0.00025), metrics=['mae'])

dqn.load_weights('./drive/ProX/dqn_eps2_{}_weights.h5f'.format(ENV_NAME))
dqn.test(env, nb_episodes=5, visualize=False)

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_6 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_7 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
acti

In [0]:
import numpy as np
import gym
import keras
#import pydot

from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

from keras import backend as K # 柴田

folder = './drive/ProX/mario6_log3'


ENV_NAME = 'SuperMarioBros-v0'

INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class GameProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, folder, force=False, video_callable=(lambda ep: ep % 10 == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

#print(env.observation_space.shape) #(240, 256, 3)

input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add( Permute((2,3,1), input_shape=input_shape ) )
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

#plot_model(model, to_file='./drive/ProX/model.png')

memory = SequentialMemory(limit=300000, window_length=WINDOW_LENGTH)
processor = GameProcessor()
policy = MaxBoltzmannQPolicy(eps=0.8)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=500, 
             processor=processor, target_model_update=10000, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.load_weights('./drive/ProX/dqn_epsbol4_{}_weights.h5f'.format(ENV_NAME))
checkpoint_weights_filename = folder + '/dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = folder + '/dqn_{}_log.json'.format(ENV_NAME)

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env,callbacks=callbacks, nb_steps=300000, visualize=False, verbose=2)
dqn.save_weights('./drive/ProX/dqn_epsbol4_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_6 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_7 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
acti

### カラーで実行
遅すぎ

In [0]:
import numpy as np
import gym
import keras
#import pydot

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

from keras import backend as K # 柴田


ENV_NAME = 'SuperMarioBros-v0'

# Get the environment and extract the number of actions.
env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, './drive/ProX/mario3', force=False, video_callable=(lambda ep: ep % 10 == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

#print(env.observation_space.shape) #(240, 256, 3)

model = Sequential()
model.add( Permute((2,3,4,1), input_shape=(4, 240, 256, 3) ) )
model.add( Reshape( (240, 256, 12) ) )
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
#model.build()
print(model.summary())

#plot_model(model, to_file='./drive/ProX/model.png')

memory = SequentialMemory(limit=30000, window_length=4)
#policy = BoltzmannQPolicy()
#policy = EpsGreedyQPolicy()
policy = MaxBoltzmannQPolicy(eps=.8)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=200, 
             target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.load_weights('./drive/ProX/dqn_epsbol_{}_weights.h5f'.format(ENV_NAME))
checkpoint_weights_filename = './drive/ProX/dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = './drive/ProX/mario3/dqn_{}_log.json'.format(ENV_NAME)

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env,callbacks=callbacks, nb_steps=50000, visualize=False, verbose=2)
dqn.save_weights('./drive/ProX/dqn_epsbol_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 240, 256, 3, 4)    0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 240, 256, 12)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 59, 63, 32)        24608     
_________________________________________________________________
activation_1 (Activation)    (None, 59, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 30, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 30, 64)        0         
_________________________________________________________________
conv

In [0]:
import numpy as np
import gym
import keras
#import pydot

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Reshape, MaxPooling2D, Permute
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import plot_model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers

from keras import backend as K # 柴田


ENV_NAME = 'SuperMarioBros-v0'

# Get the environment and extract the number of actions.
env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, './drive/ProX/mario3', force=False, video_callable=(lambda ep: ep % 1 == 0))

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

#print(env.observation_space.shape) #(240, 256, 3)

model = Sequential()
model.add( Permute((2,3,4,1), input_shape=(4, 240, 256, 3) ) )
model.add( Reshape( (240, 256, 12) ) )
model.add(Conv2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
#model.build()
print(model.summary())

#plot_model(model, to_file='./drive/ProX/model.png')

memory = SequentialMemory(limit=30000, window_length=4)
#policy = BoltzmannQPolicy()
#policy = EpsGreedyQPolicy()
policy = MaxBoltzmannQPolicy(eps=.8)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=200, 
             target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.load_weights('./drive/ProX/dqn_epsbol_{}_weights.h5f'.format(ENV_NAME))
checkpoint_weights_filename = './drive/ProX/dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = './drive/ProX/mario3/dqn_{}_log.json'.format(ENV_NAME)

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.test(env, nb_episodes=5, visualize=False)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 240, 256, 3, 4)    0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 240, 256, 12)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 59, 63, 32)        24608     
_________________________________________________________________
activation_1 (Activation)    (None, 59, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 30, 64)        32832     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 30, 64)        0         
_________________________________________________________________
conv

In [0]:
!apt-get -qq install -y graphviz && pip install -q pydot

## 前のやつ

In [0]:
import numpy as np
import gym
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers # <=追加


ENV_NAME = 'SuperMarioBros-v0'

# Get the environment and extract the number of actions.
env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, './drive/mario', force=True, video_callable=(lambda ep: ep % 10 == 0))  # <=追加

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
             target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.load_weights('./drive/dqn_{}_weights.h5f'.format(ENV_NAME))
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(ENV_NAME)

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env,callbacks=callbacks, nb_steps=400000, visualize=False, verbose=2)
dqn.save_weights('./drive/dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

In [0]:
!cp dqn_SuperMarioBros-v0_log.json ./drive/

In [0]:
from google.colab import files
files.download("dqn_SuperMarioBros-v0_log.json")

In [0]:
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
from keras.utils import plot_model

In [0]:
from keras.utils.visualize_util import plot
plot(dqn, to_file="model.png", show_shapes=True)

ModuleNotFoundError: ignored

In [0]:
env = wrappers.Monitor(env, './drive/mario2', force=True, video_callable=(lambda ep: ep % 1 == 0))  # <=追加
dqn.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))
dqn.test(env, nb_episodes=5, visualize=True)

NameError: ignored

In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from gym import wrappers # <=追加

ENV_NAME = 'SuperMarioBros-v0'

# Get the environment and extract the number of actions.
env = gym_super_mario_bros.make(ENV_NAME)
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

env = wrappers.Monitor(env, './drive/mario2', force=True, video_callable=(lambda ep: ep % 1 == 0))  # <=追加

np.random.seed(123)
env.seed(123)
nb_actions = len(SIMPLE_MOVEMENT)

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=0,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.load_weights('./drive/dqn_{}_weights.h5f'.format(ENV_NAME))
dqn.test(env, nb_episodes=5, visualize=False)

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: Could not seed environment <SuperMarioBrosEnv<SuperMarioBros-v0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                2949136   
_________________________________________________________________
activation_5 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_6 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
acti

In [0]:
env = wrappers.Monitor(env, './drive/mario2', force=True, video_callable=(lambda ep: ep % 1 == 0))  # <=追加

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=False)


Testing for 5 episodes ...


TypeError: ignored

In [0]:
import rl.callbacks
class EpisodeLogger(rl.callbacks.Callback):
    def __init__(self):
        self.observations = {}
        self.rewards = {}
        self.actions = {}

    def on_episode_begin(self, episode, logs):
        self.observations[episode] = []
        self.rewards[episode] = []
        self.actions[episode] = []

    def on_step_end(self, step, logs):
        episode = logs['episode']
        self.observations[episode].append(logs['observation'])
        self.rewards[episode].append(logs['reward'])
        self.actions[episode].append(logs['action'])

env = wrappers.Monitor(env, './drive/mario2', force=True, video_callable=(lambda ep: ep % 1 == 0))
cb_ep = EpisodeLogger()
dqn.test(env, nb_episodes=10, visualize=False, callbacks=[cb_ep])


%matplotlib inline
import matplotlib.pyplot as plt

for obs in cb_ep.observations.values():
    plt.plot([o[0] for o in obs])
plt.xlabel("step")
plt.ylabel("pos")

Testing for 10 episodes ...


Error: ignored

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>